In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Dense, Flatten, Dropout, GlobalAveragePooling1D
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelBinarizer

# Load the preprocessed data
X_train = np.loadtxt("../Datasets/CAPP Dataset/SubjectIndependent50PercentOverlap/X_train.txt")
y_train = np.loadtxt("../Datasets/CAPP Dataset/SubjectIndependent50PercentOverlap/y_train.txt")
X_test = np.loadtxt("../Datasets/CAPP Dataset/SubjectIndependent50PercentOverlap/X_test.txt")
y_test = np.loadtxt("../Datasets/CAPP Dataset/SubjectIndependent50PercentOverlap/y_test.txt")

# Reshape the input data for Conv1D layers
n_timestep = 100
n_features = 9
X_train = X_train.reshape(X_train.shape[0], n_timestep, n_features)
X_test = X_test.reshape(X_test.shape[0], n_timestep, n_features)

# One-hot encode the labels
lb = LabelBinarizer()
y_train = lb.fit_transform(y_train)
y_test = lb.transform(y_test)

def create_model(n_timesteps, n_features, n_outputs):
    model = Sequential([
        Conv1D(filters=64, kernel_size=3, activation="relu", input_shape=(n_timesteps, n_features)),
        Conv1D(filters=64, kernel_size=3, activation="relu"),
        GlobalAveragePooling1D(),
        Dense(128, activation="relu"),
        Dropout(0.5),
        Dense(64, activation="relu"),
        Dense(n_outputs, activation="softmax")
    ])
    return model

n_outputs = y_train.shape[1]
conv_mlp = create_model(n_timestep, n_features, n_outputs)
conv_mlp.compile(optimizer=Adam(learning_rate=0.001), loss="categorical_crossentropy", metrics=["accuracy"])

# Train the model
history = conv_mlp.fit(X_train, y_train, epochs=200, batch_size=512, validation_split=0.2, verbose=1)

# Evaluate the model
y_pred = conv_mlp.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true_classes = np.argmax(y_test, axis=1)

# Print classification report
print(classification_report(y_true_classes, y_pred_classes, digits=5))

2024-09-27 06:24:15.153295: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-27 06:24:15.164620: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-27 06:24:15.168015: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-27 06:24:15.177553: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-27 06:24:15.727866: W tensorflow/compiler/tf2

Epoch 1/100


I0000 00:00:1727396678.729080  104900 service.cc:146] XLA service 0x7d35a0006330 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1727396678.729126  104900 service.cc:154]   StreamExecutor device (0): NVIDIA GeForce RTX 3070, Compute Capability 8.6
2024-09-27 06:24:38.763638: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-09-27 06:24:38.876737: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN version 8907


 119/2495 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.1245 - loss: 2.8543

I0000 00:00:1727396680.498315  104900 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


2495/2495 ━━━━━━━━━━━━━━━━━━━━ 8s 2ms/step - accuracy: 0.5029 - loss: 1.5353 - val_accuracy: 0.7129 - val_loss: 0.9150
Epoch 2/100
2495/2495 ━━━━━━━━━━━━━━━━━━━━ 2s 928us/step - accuracy: 0.7683 - loss: 0.6954 - val_accuracy: 0.7322 - val_loss: 0.9414
Epoch 3/100
2495/2495 ━━━━━━━━━━━━━━━━━━━━ 2s 993us/step - accuracy: 0.8130 - loss: 0.5590 - val_accuracy: 0.7482 - val_loss: 1.0235
Epoch 4/100
2495/2495 ━━━━━━━━━━━━━━━━━━━━ 2s 895us/step - accuracy: 0.8391 - loss: 0.4845 - val_accuracy: 0.7556 - val_loss: 0.9666
Epoch 5/100
2495/2495 ━━━━━━━━━━━━━━━━━━━━ 2s 895us/step - accuracy: 0.8562 - loss: 0.4325 - val_accuracy: 0.7414 - val_loss: 0.9636
Epoch 6/100
2495/2495 ━━━━━━━━━━━━━━━━━━━━ 2s 923us/step - accuracy: 0.8646 - loss: 0.4020 - val_accuracy: 0.7636 - val_loss: 1.0276
Epoch 7/100
2495/2495 ━━━━━━━━━━━━━━━━━━━━ 2s 895us/step - accuracy: 0.8753 - loss: 0.3745 - val_accuracy: 0.7445 - val_loss: 1.0201
Epoch 8/100
2495/2495 ━━━━━━━━━━━━━━━━━━━━ 2s 968us/step - accuracy: 0.8856 - loss:

In [ ]:
# Convert the model to TensorFlow Lite format
converter = tf.lite.TFLiteConverter.from_keras_model(conv_mlp)
tflite_model = converter.convert()

# Save the TFLite model
with open("GeneratedTFLiteFilesAndOGModels/conv_mlp_model.tflite", "wb") as f:
    f.write(tflite_model)

print("TensorFlow Lite model has been saved as 'conv_mlp_model.tflite'")

# Optional: Save the Keras model
conv_mlp.save("conv_mlp_model.h5")
print("Keras model has been saved as 'conv_mlp_model.h5'")

# If you still need to generate a C header file
try:
    from everywhereml.code_generators.tensorflow import convert_model

    c_header = convert_model(conv_mlp, X_test, y_test, model_name='conv_mlp_model')

    with open("GeneratedHeaderFiles/conv_mlp_model.h", "w") as file:
        file.write(c_header)

    print("C header file has been saved as GeneratedHeaderFiles/conv_mlp_model.h'")
except ImportError:
    print("everywhereml library not found. Skipping C header file generation.")


In [5]:
# Testing out the TFLite model when optimized for normal edge devices
from sklearn.metrics import accuracy_score

# Loading the test data
X_test = np.loadtxt("../Datasets/CAPP Dataset/SubjectIndependent50PercentOverlap/X_test.txt")
y_test = np.loadtxt("../Datasets/CAPP Dataset/SubjectIndependent50PercentOverlap/y_test.txt")

# Reshaping the input data
n_timestep = 100
n_features = 9
X_test = X_test.reshape(X_test.shape[0], n_timestep, n_features)

# Doing One-hot encode the labels
lb = LabelBinarizer()
y_test = lb.fit_transform(y_test)

# Loading the TFLite model
interpreter = tf.lite.Interpreter(model_path="GeneratedTFLiteFilesAndOGModels/conv_mlp_model.tflite")
interpreter.allocate_tensors()

# Allocating input and output tensors
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Test the model on random input data
input_shape = input_details[0]["shape"]
input_data = X_test

# Now running predictions on every test sample
y_pred_tflite = []
for i in range(len(input_data)):
    input_tensor = np.array(input_data[i], dtype=np.float32)
    input_tensor = np.expand_dims(input_tensor, axis=0)
    interpreter.set_tensor(input_details[0]["index"], input_tensor)
    interpreter.invoke()
    output_data = interpreter.get_tensor(output_details[0]["index"])
    y_pred_tflite.append(output_data)

y_pred_tflite = np.vstack(y_pred_tflite)

# Converting predictions to class labels
y_pred_classes_tflite = np.argmax(y_pred_tflite, axis=1)
y_true_classes = np.argmax(y_test, axis=1)

# Calculating the accuracy
accuracy_tflite = accuracy_score(y_true_classes, y_pred_classes_tflite)
print(f"TFLite Model Accuracy: {accuracy_tflite:.5f}")

print("\nClassification Report:")
print(classification_report(y_true_classes, y_pred_classes_tflite, digits=5))

try:
    original_model = tf.keras.models.load_model("GeneratedTFLiteFilesAndOGModels/conv_mlp_model.h5")
    y_pred_original = original_model.predict(X_test)
    y_pred_classes_original = np.argmax(y_pred_original, axis=1)
    accuracy_original = accuracy_score(y_true_classes, y_pred_classes_original)
    print(f"\nOriginal Keras Model Accuracy: {accuracy_original:.5f}")
    
    prediction_match = np.mean(y_pred_classes_tflite == y_pred_classes_original)
    print(f"\nPrediction Match between TFLite and Original model: {prediction_match:.5f}")
except:
    print("\nError Check your path")

TFLite Model Accuracy: 0.75816

Classification Report:
              precision    recall  f1-score   support

           0    0.86392   0.88350   0.87360       927
           1    0.98242   0.96440   0.97333       927
           2    0.83250   0.89827   0.86413       924
           3    0.94359   0.83922   0.88835       877
           4    0.78603   0.74947   0.76732       946
           5    0.73430   0.69681   0.71507       940
           6    0.85089   0.90678   0.87795       944
           7    0.76979   0.89053   0.82577       950
           8    0.87473   0.42587   0.57284       951
           9    0.68642   0.38727   0.49517       927
          10    0.83883   0.69289   0.75891      1322
          11    0.60742   0.95745   0.74329       564
          12    0.82678   0.94641   0.88256       933
          13    0.63463   0.96146   0.76458       934
          14    0.70716   0.82979   0.76358       940
          15    0.63844   0.63362   0.63602       928
          16    0.98759   